In [7]:
import asyncio
import nest_asyncio
import json
import sys
from airstack.execute_query import AirstackClient

nest_asyncio.apply()

# Load API key from config.json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
    api_key = config['airstack_api_key']

api_client = AirstackClient(api_key=api_key)

query = """
query MyQuery {
  TrendingCasts(
    input: {timeFrame: one_day, blockchain: ALL, criteria: social_capital_value, limit: 1}
  ) {
    TrendingCast {
      cast {
        castedAtTimestamp
        embeds
        hash
        fid
        text
        numberOfRecasts
        numberOfLikes
        numberOfReplies
        castedBy {
          fnames
        }
        channel {
          channelId
          name
        }
        socialCapitalValue{
          formattedValue
          rawValue
        }
      }
    }
  }
}
"""

async def main():
    all_data = []
    execute_query_client = api_client.create_execute_query_object(query=query)
    query_response = await execute_query_client.execute_paginated_query()
    
    if query_response.error is not None:
        raise Exception(query_response.error.message)
    
    all_data.extend(query_response.data['TrendingCasts']['TrendingCast'])
    print(query_response.data)  # Printing initial page data
    
    page_limit = 5  # Set the maximum number of pages to fetch
    current_page = 0  # Initialize page counter
    
    while query_response.has_next_page and current_page < page_limit:
        print("Fetching next page...")
        query_response = await query_response.get_next_page
        if query_response.data:
            all_data.extend(query_response.data['TrendingCasts']['TrendingCast'])
            print(query_response.data)  # Printing data of each subsequent page
        current_page += 1  # Increment the page counter
    
    with open('AllTrendOutput.txt', 'w') as file:
        json.dump(all_data, file, indent=4)

asyncio.run(main())

{'TrendingCasts': {'TrendingCast': [{'cast': {'castedAtTimestamp': '2024-06-24T18:53:47Z', 'embeds': [{'castId': {'fid': 244917, 'hash': '0x717b6faf943b18039ada2e834394f957200b6065'}}], 'hash': '0xeb35abf561c42ecc85f394255bda9bdc2ea5a2bb', 'fid': '1766', 'text': "We've only just started building crucial long-term incentives for core contributors\n\nIMO needs another order of magnitude flowing through this holistic, credibly neutral, member-governed funding mech.\n\n(incredible work to squeeze this viz out of Dune by @cheeky-gorilla 🔥)", 'numberOfRecasts': 62, 'numberOfLikes': 266, 'numberOfReplies': 15, 'castedBy': {'fnames': ['trent']}, 'channel': None, 'socialCapitalValue': {'formattedValue': 12476.571793589805, 'rawValue': '1247657179358943'}}}], 'pageInfo': {'nextCursor': 'MQ==', 'prevCursor': ''}}}
Fetching next page...
{'TrendingCasts': {'TrendingCast': [{'cast': {'castedAtTimestamp': '2024-06-24T14:15:54Z', 'embeds': [{'castId': {'fid': 5179, 'hash': '0x2221a9b8ea4b9858596975aa0